In [2]:
print('hello')

hello


In [4]:
!which python # ! 하면 환경 알 수 있음

/opt/conda/bin/python


In [9]:
!python --version # python 버전 확인

Python 3.11.6


In [10]:
!java -version # java 버전 확인

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [11]:
!which java

/usr/bin/java


In [13]:
pip show pyspark # pyspark 설명 확인(버전 문제 확인하기)

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [15]:
#SPARK_HOME
import os
os.environ.get('SPARK_HOME')

'/usr/local/spark'

In [18]:
# JAVA_HOME
import os
os.environ.get('JAVA_HOME')

In [21]:
# PYTHONPATH
import os
os.environ.get('PYTHONPATH')

'/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python:'

In [23]:
import pyspark #spark을 python으로 만들어놓은 라이브러리 

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() # context를 만들어 주는 애 builder. chaining
# SparkContext.SparkSession

In [25]:
spark

In [26]:
spark.stop()

In [27]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [28]:
spark

In [32]:
data = [('Alice',1), ('Bob',2), ('Charlie',3)]
type(data) # 파이썬 데이터 타입-리스트

list

In [38]:
# spark이 알고 있는 데이터타입으로 변환
# DataFrame 객체(분산객체)를 생성 <> 판다스의 데이터프레임이 아님.
data1 = spark.createDataFrame(data, ['Name','Value']) # spark의 객체 dataframe
data1

DataFrame[Name: string, Value: bigint]

In [45]:
data1[1][1]

Column<'Value[1]'>

In [44]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



# RDD 객체 생성

In [47]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() # 기존에 있으면 그냥 줌. 없으면 생성

In [48]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5]) # 병렬로 관리됨 parallelize.데이터를 직접적으로 명시적으로 주고 있음
rdd

ParallelCollectionRDD[36] at readRDDFromFile at PythonRDD.scala:289

In [49]:
data1

DataFrame[Name: string, Value: bigint]

In [50]:
rdd.show() # show는 데이터프레임의 함수!!

AttributeError: 'RDD' object has no attribute 'show'

In [54]:
rdd.take(5) # rdd의 객체를 출력하는 함수. n개를 무조건 지정해야함!!!!!

[1, 2, 3, 4, 5]

In [52]:
# map 연산: rdd값으로 연산
squared_rdd = rdd.map(lambda x:x*x,)
squared_rdd # 연산 결과는 spark context에서  보이지 않음

PythonRDD[39] at RDD at PythonRDD.scala:53

In [53]:
squared_rdd.take(3) # 연산결과를 새로운 rdd로 생성. 기존의 rdd는 그대로 있음

[1, 4, 9]

In [55]:
squared_rdd.collect() # 전체값을 출력

[1, 4, 9, 16, 25]

In [59]:
data1.filter(data1.Name == "Bob").show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [60]:
data1.filter(data1.Value > 2).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [61]:
data1.createOrReplaceTempView('people') # 임시 뷰

In [63]:
spark.sql('select * from people').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [65]:
spark.sql('select * from people where Name == "Bob" ').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [64]:
spark.sql('select * from people where Value > 2').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



# MLlib

In [67]:
from pyspark.ml.regression import LinearRegression

In [68]:
# array 형태로 만든 feature, target의 데이터를 사용하기 위해
import numpy as np

In [69]:
from pyspark.ml.feature import VectorAssembler

In [71]:
data_age = [('Alice',25), ('Bob',30), ('Charlie',33)]
data2 = spark.createDataFrame(data_age, ['Name','Age'])
data2 # 데이터 속성 2개 name, age

DataFrame[Name: string, Age: bigint]

In [72]:
assemblear = VectorAssembler(inputCols = ['Age'], outputCol='features') # 하나로 묶어주는 어셈블러
vector_df = assemblear.transform(data2)
vector_df

DataFrame[Name: string, Age: bigint, features: vector]

In [73]:
lr = LinearRegression(featuresCol='features', labelCol = 'Age') # linearregression 객체
model = lr.fit(vector_df)

In [74]:
pred = model.transform(vector_df) # 예측
pred

DataFrame[Name: string, Age: bigint, features: vector, prediction: double]

In [75]:
pred.show()

+-------+---+--------+-----------------+
|   Name|Age|features|       prediction|
+-------+---+--------+-----------------+
|  Alice| 25|  [25.0]|24.99999999999993|
|    Bob| 30|  [30.0]|30.00000000000001|
|Charlie| 33|  [33.0]|33.00000000000006|
+-------+---+--------+-----------------+



In [78]:
spark.stop()

# Streaming

In [77]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [79]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() # chaining

In [ ]:
lines = spark.readStream.format('socket')\
    .option('host', 'localhost')\
    .option('port',9999)\
    .load() # STREAMMING DATA

In [ ]:
words = lines.select(explode(split(lines.value, ' ')).alias('word')
# 스트리밍에 있는 데이터를 공백을 기준으로 자르고 'word'라는 이름의 데이터 프레임으로 만들기(aaaa bbbb cccc hello)

In [80]:
spark.stop()